In [1]:
import torch 
import json
import numpy as np
import pandas as pd
import custom_utils

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device("cuda")

In [2]:
from sklearn.model_selection import train_test_split

# read
sentences, speakers, labels = custom_utils.read_data("training", "training_labels.json")

# split
df = pd.DataFrame({"sentences" : sentences, "speakers" : speakers, "labels" : labels})

train, test = train_test_split(df, test_size=0.2, random_state=69, stratify=df.labels)

train, valid = train_test_split(train, test_size=0.3, random_state=69, stratify=train.labels)


print(f"Train: {len(train)}\nTest: {len(test)}\nValid: {len(valid)}")
train.head()

Train: 40668
Test: 14525
Valid: 17430


,sentences,speakers,labels
70575,Yeah I think it should be a little distinct fr...,PM,0
19197,"Okay , my turn .",ID,0
32883,but when we want a scrolling wheel w we also n...,PM,1
66445,Yeah .,UI,0
67299,and it still looks very fancy .,PM,0


In [3]:
from sentence_transformers import SentenceTransformer

# hot encoder for speakers
switcher = {
    "PM" : [1,0,0,0],
    "ME" : [0,1,0,0],
    "UI" : [0,0,1,0],
    "ID" : [0,0,0,1]
}

# embed
bert = SentenceTransformer('all-MiniLM-L6-v2')

train_sentences = bert.encode(train['sentences'].to_numpy(), convert_to_tensor=True, show_progress_bar=True).to(device)
train_speaker = torch.Tensor([switcher[el] for el in train['speakers']]).to(device)
train_len = torch.Tensor([[len(sentence.split())] for sentence in train['sentences']]).to(device)
train_X = torch.cat((train_sentences, train_speaker, train_len), dim=1)
train_y = torch.tensor(train['labels'].to_numpy())

valid_sentences = bert.encode(valid['sentences'].to_numpy(), convert_to_tensor=True, show_progress_bar=True).to(device)
valid_speaker = torch.Tensor([switcher[el] for el in valid['speakers']]).to(device)
valid_len = torch.Tensor([[len(sentence.split())] for sentence in valid['sentences']]).to(device)
valid_X = torch.cat((valid_sentences, valid_speaker, valid_len), dim=1)
valid_y = torch.tensor(valid['labels'].to_numpy())

test_sentences = bert.encode(test['sentences'].to_numpy(), convert_to_tensor=True, show_progress_bar=True).to(device)
test_speaker = torch.Tensor([switcher[el] for el in test['speakers']]).to(device)
test_len = torch.Tensor([[len(sentence.split())] for sentence in test['sentences']]).to(device)
test_X = torch.cat((test_sentences, test_speaker, test_len), dim=1)
test_y = torch.tensor(test['labels'].to_numpy())

c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 454/454 [00:19<00:00, 22.86it/s]


In [13]:
from torch.utils.data import TensorDataset, DataLoader

def data_loader(batch_size):
    # create tensor datasets
    trainset = TensorDataset((train_X).to(device), (train_y).to(device))
    validset = TensorDataset((valid_X).to(device), (valid_y).to(device))
    testset = TensorDataset((test_X).to(device), (test_y).to(device))

    # create dataloaders
    train_loader = DataLoader(trainset, shuffle=True, batch_size=batch_size, generator=torch.Generator(device=device))
    valid_loader = DataLoader(validset, shuffle=True, batch_size=batch_size, generator=torch.Generator(device=device))
    test_loader = DataLoader(testset, shuffle=True, batch_size=batch_size, generator=torch.Generator(device=device))
    
    return train_loader, valid_loader, test_loader

In [14]:
def MLP(params):
    # Model
    n_layers = params['n_layers']
    layers = []

    in_features = params['input_size']
    for i in range(n_layers):
        out_features = params[f'n_{i}_size']
        layers.append(torch.nn.Linear(in_features, out_features))
        layers.append(torch.nn.ReLU())
        
        # suggest dropout
        p = params['n_p']
        layers.append(torch.nn.Dropout(p))

        # updating next layer size
        in_features = out_features
        
    layers.append(torch.nn.Linear(in_features, params['output_size']))
    model = torch.nn.Sequential(*layers)
    return model

In [12]:
# from sklearn.utils.class_weight import compute_class_weight

# class_weights = compute_class_weight('balanced', classes=np.unique(train['labels'].to_numpy()), y=train['labels'].to_numpy())

# criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights).float()) 


In [6]:
import optuna
from torchmetrics.classification import F1Score
from sklearn.utils.class_weight import compute_class_weight

def train_MLP(trial):
    
    params = {
        "n_layers" : trial.suggest_int("n_layers", 3, 7),
        "input_size" : int(train_X.shape[1]),
        "output_size" : 2,
        "n_p" : trial.suggest_float("n_p", 0.4, 0.8),
        "lr" : trial.suggest_float("lr", 1e-4, 1e-3),
        "weight_decay" : trial.suggest_float("weight_decay", 1e-5, 1e-4),
        "batch_size" : trial.suggest_int("batch_size", 400, 600)
    }
    for i in range(trial.params["n_layers"]):
        params[f"n_{i}_size"] = trial.suggest_int(f"n_{i}_size", 200, 800)

    
    model = MLP(params)
    class_weights = compute_class_weight('balanced', classes=np.unique(train['labels'].to_numpy()), y=train['labels'].to_numpy())
    criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights).float()) 
    # criterion = torch.nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"], weight_decay=params["weight_decay"])
    f1 = F1Score(task='binary', num_classes=params['output_size']).to(device)

    train_loader, valid_loader, test_loader = data_loader(params["batch_size"])
    
    n_epochs = 20
    it = 0
    hst_train_loss = [] 
    hst_valid_loss = []
    hst_f1_score = []

    best_valid_loss = float("inf")
    patience = 10
    for epoch in range(n_epochs):
        if patience == 0: break
        for samples, labels in train_loader:
            if patience == 0: break
            it += 1

            # train step
            model.train()
            optimizer.zero_grad()
            out = model(samples)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            

            if it % 100 == 0:
                model.eval()

                train_loss = loss.cpu().detach().numpy() / 1
                valid_loss = 0
                f1_score = 0
                for samples, labels in valid_loader:
                    out = model(samples)
                    loss = criterion(out, labels)
                    valid_loss += loss.cpu().detach().numpy() / len(valid_loader)
                    f1_score += f1(labels, out.argmax(dim=1)).cpu().detach().numpy() / len(valid_loader)
                
                # early stopping
                if valid_loss < best_valid_loss:
                    best_valid_loss = valid_loss
                    best_weights = model.state_dict()
                    patience = 10
                else:
                    patience -= 1 
                
                hst_train_loss.append(train_loss)
                hst_valid_loss.append(valid_loss)
                hst_f1_score.append(f1_score)

                print('Iter: {} | Train Loss: {} | Val Loss: {} | F1-score: {}'.format(it, train_loss, valid_loss, f1_score))
    
    # objective function criterion
    combined = sorted(zip(hst_valid_loss, hst_f1_score), key=lambda x : x[0])
    _, scores = zip(*combined)
    qtd = 3
    final_score = sum(scores[:qtd]) / qtd

    torch.save(best_weights, f"models/mlp_{trial.number}.pt")
    results = {
        "score" : final_score,
        "params" : params,
        "valid_loss" : hst_valid_loss,
        "train_loss" : hst_train_loss,
        "f1_score" : hst_f1_score, 
    }
    json.dump(results, open(f"models/mlp_results_{trial.number}.json", "w"))

    return final_score

In [7]:
study = optuna.create_study(direction='maximize')
study.optimize(train_MLP, n_trials=500)

[I 2023-12-04 04:20:51,893] A new study created in memory with name: no-name-ef86ba72-dd20-4c8c-9e04-48d7bed8f249


Iter: 100 | Train Loss: 0.36012235283851624 | Val Loss: 0.3319387937846937 | F1-score: 0.43481223206771036
Iter: 200 | Train Loss: 0.3128753900527954 | Val Loss: 0.3253439374660191 | F1-score: 0.46063435940366043
Iter: 300 | Train Loss: 0.29471731185913086 | Val Loss: 0.32281238860205597 | F1-score: 0.4495595034800079
Iter: 400 | Train Loss: 0.28942471742630005 | Val Loss: 0.32024350291804266 | F1-score: 0.5261493419346057
Iter: 500 | Train Loss: 0.3100268244743347 | Val Loss: 0.32028789504578237 | F1-score: 0.5242662116100913
Iter: 600 | Train Loss: 0.3141099214553833 | Val Loss: 0.31767960833875764 | F1-score: 0.48578170804600956
Iter: 700 | Train Loss: 0.3051414489746094 | Val Loss: 0.31737125233600005 | F1-score: 0.5308804135573537
Iter: 800 | Train Loss: 0.3383527398109436 | Val Loss: 0.31764967818009215 | F1-score: 0.5351550751610806
Iter: 900 | Train Loss: 0.29451027512550354 | Val Loss: 0.3179136436236532 | F1-score: 0.48060188167973583
Iter: 1000 | Train Loss: 0.30503761768341

[I 2023-12-04 04:21:13,925] Trial 0 finished with value: 0.526072474948147 and parameters: {'n_layers': 4, 'n_p': 0.6130724483887233, 'lr': 0.0005326806903942525, 'weight_decay': 1.9472376819179503e-05, 'batch_size': 465, 'n_0_size': 262, 'n_1_size': 476, 'n_2_size': 734, 'n_3_size': 406}. Best is trial 0 with value: 0.526072474948147.


Iter: 100 | Train Loss: 0.3177231252193451 | Val Loss: 0.3285485692322254 | F1-score: 0.3658629674464464
Iter: 200 | Train Loss: 0.3238745927810669 | Val Loss: 0.3248340729624033 | F1-score: 0.46205563098192215
Iter: 300 | Train Loss: 0.3223522901535034 | Val Loss: 0.3208871800452471 | F1-score: 0.4348042318597436
Iter: 400 | Train Loss: 0.29421064257621765 | Val Loss: 0.3199586998671293 | F1-score: 0.41932765301316977
Iter: 500 | Train Loss: 0.31475672125816345 | Val Loss: 0.3177069555968046 | F1-score: 0.4992232220247388
Iter: 600 | Train Loss: 0.34188562631607056 | Val Loss: 0.317690746858716 | F1-score: 0.5278472304344177
Iter: 700 | Train Loss: 0.2924685776233673 | Val Loss: 0.3164311954751611 | F1-score: 0.519834122620523
Iter: 800 | Train Loss: 0.3139229714870453 | Val Loss: 0.3164797741919756 | F1-score: 0.5457660825923085
Iter: 900 | Train Loss: 0.33876001834869385 | Val Loss: 0.3178023360669613 | F1-score: 0.556452208198607
Iter: 1000 | Train Loss: 0.2849796712398529 | Val Lo

[I 2023-12-04 04:21:39,015] Trial 1 finished with value: 0.5311491452157497 and parameters: {'n_layers': 5, 'n_p': 0.47246377406868323, 'lr': 0.0004275749981812202, 'weight_decay': 6.54390352066903e-05, 'batch_size': 546, 'n_0_size': 644, 'n_1_size': 292, 'n_2_size': 770, 'n_3_size': 504, 'n_4_size': 629}. Best is trial 1 with value: 0.5311491452157497.


...

Iter: 1500 | Train Loss: 0.30434882640838623 | Val Loss: 0.3196881016095479 | F1-score: 0.5368295248710747
Iter: 100 | Train Loss: 0.3380526304244995 | Val Loss: 0.3284001883232232 | F1-score: 0.4384218761415193
Iter: 200 | Train Loss: 0.3659990131855011 | Val Loss: 0.3223878172310916 | F1-score: 0.3840836349761847
Iter: 300 | Train Loss: 0.3056815564632416 | Val Loss: 0.3210319768298757 | F1-score: 0.45552724238597986
Iter: 400 | Train Loss: 0.3290694057941437 | Val Loss: 0.3203299388741002 | F1-score: 0.4449732077844215
Iter: 500 | Train Loss: 0.28578850626945496 | Val Loss: 0.31902861685463874 | F1-score: 0.4947875602678819
Iter: 600 | Train Loss: 0.30991071462631226 | Val Loss: 0.31800113392598706 | F1-score: 0.4569636556235226
Iter: 700 | Train Loss: 0.3452291786670685 | Val Loss: 0.3167224932800641 | F1-score: 0.532166159514225
Iter: 800 | Train Loss: 0.3557697832584381 | Val Loss: 0.31654421127203736 | F1-score: 0.5454061428705853
Iter: 900 | Train Loss: 0.34891560673713684 | Va

[I 2023-12-04 04:32:29,212] Trial 22 finished with value: 0.545169672279647 and parameters: {'n_layers': 7, 'n_p': 0.5671581365312748, 'lr': 0.0009745327633390806, 'weight_decay': 5.960318813946822e-05, 'batch_size': 538, 'n_0_size': 304, 'n_1_size': 450, 'n_2_size': 255, 'n_3_size': 721, 'n_4_size': 381, 'n_5_size': 406, 'n_6_size': 560}. Best is trial 21 with value: 0.5541569220297263.


Iter: 100 | Train Loss: 0.2883281409740448 | Val Loss: 0.32722328266789835 | F1-score: 0.25546921164758746
Iter: 200 | Train Loss: 0.3119112253189087 | Val Loss: 0.3209892434458579 | F1-score: 0.43695792459672494
Iter: 300 | Train Loss: 0.33613675832748413 | Val Loss: 0.322852183734217 | F1-score: 0.4390075677825558
Iter: 400 | Train Loss: 0.3319854438304901 | Val Loss: 0.31766541061862824 | F1-score: 0.5040416198392068
Iter: 500 | Train Loss: 0.32106688618659973 | Val Loss: 0.31940829080920063 | F1-score: 0.47608529656164117
Iter: 600 | Train Loss: 0.3242288827896118 | Val Loss: 0.3207703790357036 | F1-score: 0.5310812313710489
Iter: 700 | Train Loss: 0.32209861278533936 | Val Loss: 0.3174079473941557 | F1-score: 0.5363087394545154
Iter: 800 | Train Loss: 0.2854755222797394 | Val Loss: 0.3176196082945793 | F1-score: 0.5086246017486818
Iter: 900 | Train Loss: 0.3160615861415863 | Val Loss: 0.3183236468222834 | F1-score: 0.4873476595647874
Iter: 1000 | Train Loss: 0.30069252848625183 | 

[W 2023-12-04 04:32:58,421] Trial 23 failed with parameters: {'n_layers': 7, 'n_p': 0.5552792636373135, 'lr': 0.0009938124997151596, 'weight_decay': 3.563732372793427e-05, 'batch_size': 575, 'n_0_size': 309, 'n_1_size': 531, 'n_2_size': 252, 'n_3_size': 734, 'n_4_size': 379, 'n_5_size': 376, 'n_6_size': 556} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\astus\AppData\Local\Temp\ipykernel_10164\356372157.py", line 89, in train_MLP
    json.dump(results, open(f"models/mlp_results_{trial.number}.json", "w"))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\astus\code\extractive-summarization\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 310, in _modified_open
    return io_ope

KeyboardInterrupt: 

In [ ]:
from pathlib import Path

score = {}
for item in Path("models").iterdir():
    if not item.suffix == ".json" : continue
    
    id = item.stem[-1]
    score[id] = json.load(open(item, "r"))["score"]

sorted(score.items(), key=lambda x : x[1])
